In [ ]:
import os
from data_loader.calib.intrinsic_extrinsic_loader import IntrinsicExtrinsicLoader
from tools.utils import *

def generate_depthmap_to_kitti360(dataset_path, calib_path, platform, sequence_name, algorithm):
  ##### Set up the output data path
  kitti360_path = os.path.join(dataset_path, sequence_name, 'kitti360')

  ##### Set up the message topic list for different platforms
  # Platform
  if platform == 'handheld':
    from cfg.dataset.cfg_handheld import dataset_sensor_frameid_dict
    from cfg.dataset.cfg_handheld import dataset_rostopic_msg_frameid_dict
  elif platform == 'ugv':
    from cfg.dataset.cfg_ugv import dataset_sensor_frameid_dict
    from cfg.dataset.cfg_ugv import dataset_rostopic_msg_frameid_dict
  elif platform == 'legged':
    from cfg.dataset.cfg_legged import dataset_sensor_frameid_dict
    from cfg.dataset.cfg_legged import dataset_rostopic_msg_frameid_dict
    if sequence_name == 'legged_grass00':
      dataset_sensor_frameid_dict, dataset_rostopic_msg_frameid_dict = \
        filter_sensor('event', dataset_sensor_frameid_dict, dataset_rostopic_msg_frameid_dict)
  elif platform =='vehicle':
    from cfg.dataset.cfg_vehicle import dataset_sensor_frameid_dict
    from cfg.dataset.cfg_vehicle import dataset_rostopic_msg_frameid_dict

  ##### Set up the sensor configuration
  int_ext_loader = IntrinsicExtrinsicLoader(is_print=False)
  int_ext_loader.load_calibration(calib_path=calib_path, sensor_frameid_dict=dataset_sensor_frameid_dict)
  print('Finish loading parameters')

  ##### Create the output data path
  if platform == 'vehicle':
    flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'vehicle_frame_cam00/color_proj_image/data'))
    flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'vehicle_frame_cam01/color_proj_image/data'))
    flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'vehicle_frame_cam00/depth_image/data'))
    flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'vehicle_frame_cam01/depth_image/data'))
  else:
    flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'frame_cam00/color_proj_image/data'))
    flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'frame_cam01/color_proj_image/data'))
    flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'frame_cam00/depth_image/data'))
    flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'frame_cam01/depth_image/data'))
  print('Finish creating Folder')

  ##### Generate depth map
  # Description: Generate depth map
  import cv2
  import open3d
  from PIL import Image
  from IPython.display import display
  import matplotlib.pyplot as plt
  cmap = plt.get_cmap('jet')
  cmap_colors = cmap(np.linspace(0, 1, 31)) * 255

  for frame_id in range(100000):
    ##### Load undistroted point cloud and undistroted image
    pcd_path = os.path.join(kitti360_path, 'ouster00_undistort/points/data', '{:06d}.pcd'.format(frame_id))
    if not os.path.exists(pcd_path):
      break

    xyz_points = np.asarray(open3d.io.read_point_cloud(pcd_path).points)
    if platform == 'vehicle':
      img_path = os.path.join(kitti360_path, 'vehicle_frame_cam00', 'image/data', '{:06d}.png'.format(frame_id)) # images are already undistorted, but not rectified
      img = cv2.imread(img_path)
      camera = int_ext_loader.sensor_collection['vehicle_frame_left_camera']
    else:
      img_path = os.path.join(kitti360_path, 'frame_cam00', 'image/data', '{:06d}.png'.format(frame_id)) # images are already undistorted, but not rectified
      img = cv2.imread(img_path)
      camera = int_ext_loader.sensor_collection['frame_left_camera']

    ##### Load extrinsics from the tf_graph
    T_cam_lidar = int_ext_loader.tf_graph.get_relative_transform(camera.frame_id, 'body_imu')
    xyz_points_cam = np.matmul(T_cam_lidar[:3, :3], xyz_points.T).T + T_cam_lidar[:3, 3].T

    ##### Project point cloud onto the camera frame
    for p_C in xyz_points_cam:
      flag, u_C = camera.project(p_C)
      if flag:
        i = int(min(p_C[2], 30.0))
        color = (int(cmap_colors[i, 0]), int(cmap_colors[i, 1]), int(cmap_colors[i, 2]))
        cv2.circle(img, (int(u_C[0]), int(u_C[1])), radius=1, color=color, thickness=-1)
    img_pillow = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    new_img_path = img_path.replace('image', 'color_proj_image')
    img_pillow.save(new_img_path)

    ##### Project point cloud onto the depth camera frame
    depth_img = np.zeros((camera.height, camera.width), dtype=np.uint32)
    for p_C in xyz_points_cam:
      flag, u_C = camera.project(p_C)
      if flag:
        depth_img[int(u_C[1])][int(u_C[0])] = np.uint32(p_C[2] * 1000)
    img_pillow = Image.fromarray(depth_img)
    new_img_path = img_path.replace('image', 'depth_image')
    img_pillow.save(new_img_path)

In [ ]:
from cfg.dataset.cfg_sequence import dataset_sequence_calib_used_dict

algorithms = ['fastlio2']
for sequence_name in dataset_sequence_calib_used_dict.keys():
  for algorithm in algorithms:
    platform = dataset_sequence_calib_used_dict[sequence_name][0]
    calib_folder = dataset_sequence_calib_used_dict[sequence_name][1]
    used = dataset_sequence_calib_used_dict[sequence_name][2]
    if used:
      print('Start processing platform: {} sequence: {} with algorithm_result: {}'.format(platform, sequence_name, algorithm))
      dataset_path = '/Rocket_ssd/dataset/FusionPortable_dataset_develop/sensor_data/data_refined'
      calib_path = os.path.join(dataset_path, '../../calibration_files', calib_folder, 'calib')
      generate_depthmap_to_kitti360(dataset_path, calib_path, platform, sequence_name, algorithm)
      print('Finish generating depth images')